# 延后初始化

深度学习框架无法判断网络的输入维度是什么。 这里的诀窍是框架的延后初始化（defers initialization）， 即直到数据第一次通过模型传递时，框架才会动态地推断出每个层的大小。

当使用卷积神经网络时， 由于输入维度（即图像的分辨率）将影响每个后续层的维数， 有了该技术将更加方便。 现在我们在编写代码时无须知道维度是什么就可以设置参数， 这种能力可以大大简化定义和修改模型的任务。

In [ ]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [ ]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

In [ ]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

In [ ]:
net.get_weights()

In [ ]:
# 

# 1-单轮

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key="sk-b09XXwR8nOmrdoXTrylErTOJ0mWQYxKsRZBLMmfCiV2K0grF",  # 替换为你的 Moonshot AI API Key
    # base_url="https://api.moonshot.cn/v1"
    base_url = "http://127.0.0.1:8888/v1"
)

completion = client.chat.completions.create(
    model="moonshot-v1-8k",
    messages=[
        {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一些涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"},
        {"role": "user", "content": "你好，我叫李雷，1+1 等于多少？"}
    ],
    temperature=0.3,
)

print(completion.choices[0].message.content)

- 单轮对话没有记忆能力

In [ ]:
completion = client.chat.completions.create(
    model="moonshot-v1-8k",
    messages=[
        {"role": "user", "content": "我叫什么名字？"}
    ],
    temperature=0.3,
)

print(completion.choices[0].message.content)

# 2-多轮

In [ ]:
from openai import OpenAI
import time
import pprint 
client = OpenAI(
    api_key = "sk-b09XXwR8nOmrdoXTrylErTOJ0mWQYxKsRZBLMmfCiV2K0grF", # 在这里将 MOONSHOT_API_KEY 替换为你从 Kimi 开放平台申请的 API Key
    # base_url = "https://api.moonshot.cn/v1",
	base_url = "http://127.0.0.1:8888/v1"
)
 
system_messages = [
	# {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"},
	{"role": "system", "content": "你是 Kimi， 你擅长中文。"},
]
# global messages
messages = []
# 限制上下文长度
def make_messages(input: str, n: int = 20):   
	global messages
	messages.append({
		"role": "user",
		"content": input,	
	})
 
	new_messages = []
	new_messages.extend(system_messages)
 
	if len(messages) > n:
		messages = messages[-n:]
 
	new_messages.extend(messages)
	return new_messages
 
 
def chat(input: str) -> str:
	msg = make_messages(input)
	
	completion = client.chat.completions.create(
        model="moonshot-v1-auto",
        messages=msg,
        temperature=0.3,
    )
	assistant_message = completion.choices[0].message
	messages.append(assistant_message)
	print(f"Kimi: {assistant_message.content}")
	# return assistant_message.content

In [ ]:
chat("你好，我今年 27 岁。")
time.sleep(1)
chat("你知道我今年几岁吗？")
time.sleep(1)
chat("你脾气如何？")

# Kimi Chat Assistant

In [5]:
from openai import OpenAI, OpenAIError
from pprint import pprint
import time

class KimiChatAssistant:
    def __init__(self, api_key, base_url, system_content, model, max_context_length=20, Candidates=1, use_stream=False):
        """
        初始化聊天管理器
        :param api_key: OpenAI API 密钥
        :param base_url: API 请求的基础 URL
        :param system_content: 系统消息内容
        :param model: 使用的模型名称
        :param max_context_length: 上下文最大长度，默认为 20
        """
        self.client = OpenAI(
            api_key=api_key,
            base_url=base_url
        )
        self.system_messages = [{"role": "system", "content": system_content}]
        self.messages = []
        self.model = model
        self.max_context_length = max_context_length
        self.max_attempts = 1    # 最大重试次数
        self.use_stream = use_stream
        self.Candidates = Candidates

        self.tools = [
            {
                "type": "builtin_function",
                "function": {
                    "name": "$web_search",
                },
            },
        ]


    
    def get_history(self):
        return self.system_messages + self.messages

    def reset_history(self):
        self.messages = []

    def print_context(self):
        print("当前上下文:")
        pprint.pprint(self.get_history())

    def search_impl(self, arguments):
        return arguments
 

    def prepare_messages(self, user_input):
        self.messages.append({"role": "user", "content": user_input})

        new_messages = self.system_messages.copy()
        
        if len(self.messages) > self.max_context_length:
            self.messages = self.messages[-self.max_context_length:]
        new_messages.extend(self.messages)

        return new_messages

    def chat(self, user_input):
        messages = self.prepare_messages(user_input)
        for i in range(self.max_attempts):
            try:
                completion = self.client.chat.completions.create(
                    model=self.model,
                    messages=messages,
                    temperature=0.3,
                    stream=self.use_stream,
                    n=self.Candidates,           # 返回候选
                    tools=self.tools,
                )
                if self.use_stream:
                    return completion
                assistant_message = completion.choices[0]   # 第一个候选
                self.messages.append(assistant_message.message)
                return assistant_message
            except OpenAIError as e:
                print(f"API 调用失败: {e}")
                if i < self.max_attempts - 1:
                    print(f"正在进行第 {i+1} 次重试...")
                    time.sleep(1)
            except Exception as e:
                print(f"发生未知错误: {e}")
                time.sleep(1)
                continue

        return "抱歉，系统无法提供回复，请稍后再试。"

In [6]:
if __name__ == "__main__":
    use_stream = False
    chat_manager = KimiChatAssistant(
        api_key="sk-b09XXwR8nOmrdoXTrylErTOJ0mWQYxKsRZBLMmfCiV2K0grF",
        base_url="http://127.0.0.1:8888/v1",
        # base_url = "https://api.moonshot.cn/v1",
        system_content="你是 Kimi， 你只会使用中文进行对话。",
        model="moonshot-v1-auto",
        max_context_length= 20,
        use_stream = use_stream,
        Candidates = 1,
    )

    while True:
        user_input = input("请输入你的问题(输入 'exit' 退出)：\n")
        if user_input.lower() == "exit":
            break
        response = chat_manager.chat(user_input)
        if use_stream:
            for chunk in response:
                delta = chunk.choices[0].delta
                if delta.content:
                    print(delta.content, end="", flush=True)
            print("\n")
            continue
        print(f"Kimi: {response.message.content}")
        # chat_manager.print_context()

Kimi: 嗨！你好呀，有什么可以帮你的吗？🌟
Kimi: 嗨！很高兴和你聊天。有什么我可以帮你的吗？🌷
Kimi: 你好！看起来你心情不错，喜欢打招呼呢。有什么特别的事情想要和我分享吗？或者有什么问题需要我帮忙解答？
Kimi: 嗨！看来你心情不错，喜欢重复打招呼呢。如果有什么想要聊的或者需要帮助的地方，随时告诉我哦！😄


In [ ]:
from typing import *
 
import os
import json
 
from openai import OpenAI
from openai.types.chat.chat_completion import Choice
 
client = OpenAI(
    base_url="https://api.moonshot.cn/v1",
    api_key="sk-b09XXwR8nOmrdoXTrylErTOJ0mWQYxKsRZBLMmfCiV2K0grF",
)
 
 
def search_impl(arguments: Dict[str, Any]) -> Any:
    return arguments
 
 
def chat(messages) -> Choice:
    completion = client.chat.completions.create(
        model="moonshot-v1-128k",
        messages=messages,
        temperature=0.3,
        tools=[
            {
                "type": "builtin_function",  # <-- 使用 builtin_function 声明 $web_search 函数，请在每次请求都完整地带上 tools 声明
                "function": {
                    "name": "$web_search",
                },
            }
        ]
    )
    return completion.choices[0]
 
 
def main():
    messages = [
        {"role": "system", "content": "你是 Kimi。"},
    ]
 
    # 初始提问
    messages.append({
        "role": "user",
        "content": "请搜索 Moonshot AI Context Caching 技术，并告诉我它是什么。"
    })
 
    finish_reason = None
    while finish_reason is None or finish_reason == "tool_calls":
        choice = chat(messages)
        finish_reason = choice.finish_reason
        if finish_reason == "tool_calls":  # <-- 判断当前返回内容是否包含 tool_calls
            messages.append(choice.message)  # <-- 我们将 Kimi 大模型返回给我们的 assistant 消息也添加到上下文中，以便于下次请求时 Kimi 大模型能理解我们的诉求
            for tool_call in choice.message.tool_calls:  # <-- tool_calls 可能是多个，因此我们使用循环逐个执行
                tool_call_name = tool_call.function.name
                tool_call_arguments = json.loads(tool_call.function.arguments)  # <-- arguments 是序列化后的 JSON Object，我们需要使用 json.loads 反序列化一下
                if tool_call_name == "$web_search":
                    tool_result = search_impl(tool_call_arguments)
                else:
                    tool_result = f"Error: unable to find tool by name '{tool_call_name}'"
 
                # 使用函数执行结果构造一个 role=tool 的 message，以此来向模型展示工具调用的结果；
                # 注意，我们需要在 message 中提供 tool_call_id 和 name 字段，以便 Kimi 大模型
                # 能正确匹配到对应的 tool_call。
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": tool_call_name,
                    "content": json.dumps(tool_result),  # <-- 我们约定使用字符串格式向 Kimi 大模型提交工具调用结果，因此在这里使用 json.dumps 将执行结果序列化成字符串
                })
 
    print(choice.message.content)  # <-- 在这里，我们才将模型生成的回复返回给用户
 
 
if __name__ == '__main__':
    main()

Moonshot AI 的 Context Caching（上下文缓存）技术是一种高效的数据管理技术。它允许系统预先存储那些可能会被频繁请求的大量数据或信息，这样当再次请求相同信息时，系统可以直接从缓存中快速提供，而无需重新计算或从原始数据源中检索，从而节省时间和资源。

以下是关于这项技术的要点：

1. **介绍**：上下文缓存技术可以存储大量可能会被频繁请求的数据或信息，以便快速提供，无需重新计算或检索。

2. **效果**：
   - 费用最高降低90%：特别适合频繁请求、重复引用大量初始上下文的场景，可以显著提高效率并降低费用。
   - 首Token延迟降低83%：通过上下文缓存技术，128K模型的请求首Token延迟平均可降至5秒内，降低约83%。

3. **快速开始**：
   - 创建Cache：通过API创建缓存，指定存储的数据类型和内容，并设置合适的过期时间。
   - 使用Cache：通过`role="cache"`来引用已创建好的缓存，避免将缓存内容重复添加至请求中。

4. **计费**：
   - Cache资源费由Cache创建费和Cache存储费组成。
   - 一次调用收费包括Cache调用收费、Chat未匹配Cache的Input Tokens收费和Output Tokens收费。

5. **标签系统**：上下文缓存技术还引入了标签系统，通过标签管理和使用Context Cache，降低使用过程中的心智负担。

6. **应用场景**：适用于问答机器人提供大量预设内容、对固定文档集的频繁查询、对静态代码库或知识库的定期分析、流量巨大的即时热门AI应用，以及具有复杂交互规则的代理型应用。

7. **公测时间和资格**：功能上线后，公测3个月，公测期价格可能随时调整。公测期间Context Caching功能仅开放给Tier5等级用户。

上下文缓存技术的应用可以显著降低成本、提高效率，并为各种应用场景提供更好的服务和体验。随着技术的不断发展和完善，相信这项功能将在未来发挥更加重要的作用。


# 3-工具调用

In [16]:
from openai import OpenAI
 
client = OpenAI(
    api_key = "sk-b09XXwR8nOmrdoXTrylErTOJ0mWQYxKsRZBLMmfCiV2K0grF",
    base_url = "https://api.moonshot.cn/v1",
)

tool = [
    {
      "type": "function",
      "function": {
        "name": "CodeRunner",
        "description": "代码执行器，支持运行 python 和 javascript 代码",
        "parameters": {
          "properties": {
            "language": {
              "type": "string",
              "enum": ["python", "javascript"]
            },
            "code": {
              "type": "string",
              "description": "代码写在这里"
            }
          },
          "type": "object"
        }
      }
    }
  ]

In [3]:
completion = client.chat.completions.create(
    model="moonshot-v1-8k",
    messages=[
        {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一些涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"},
        {"role": "user", "content": "编程判断 3214567 是否是素数。"}
    ],
    tools = tool,
    temperature=0.3,
)
print(completion.choices[0].message.tool_calls[0].function.arguments)

{
    "code": "def is_prime(n):\n    if n <= 1:\n        return False\n    if n <= 3:\n        return True\n    if n % 2 == 0 or n % 3 == 0:\n        return False\n    i = 5\n    while i * i <= n:\n        if n % i == 0 or n % (i + 2) == 0:\n            return False\n        i += 6\n    return True\n\n# 检查 3214567 是否是素数\nnumber_to_check = 3214567\nis_prime(number_to_check)",
    "language": "python"
}


In [4]:
completion.choices[0].message.tool_calls[0].function.arguments["code"]

TypeError: string indices must be integers